## Imports

In [1]:
# Enable automatic extension autoreloading
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import torch

from accelerate import Accelerator

from trl import set_seed, PPOTrainer

from watermark import watermark

import sys
sys.path.insert(0, str(Path.cwd().parent.resolve()))
from configs import get_script_args, get_ppo_config
from dataset import get_dataset, collator
from reward_model import get_reward_model
from model import get_model
from utils import get_tokenizer
from trainer import train

ELK_PATH = Path("/fsx/home-augustas/elk/")
modules = [
    ELK_PATH,
    ELK_PATH / "elk" / "promptsource",
]
for module in modules:
    if not str(module) in sys.path:
        sys.path.insert(0, str(module.resolve()))

print(sys.path[:2])

from templates import DatasetTemplates

['/fsx/home-augustas/elk/elk/promptsource', '/fsx/home-augustas/elk/elk/training']


In [3]:
# info = watermark(
#     packages="torch,transformers,datasets,peft,trl,tensorboard,accelerate",
#     python=True, conda=True, gpu=True,
#     current_date=True, current_time=True,
# )
# print(info)

In [4]:
# Get script args
script_args = get_script_args([
    # "--model_name=databricks/dolly-v2-3b",
    # "--tokenizer_name=databricks/dolly-v2-3b",
    "--model_name=gpt2",
    "--tokenizer_name=gpt2",
    "--reward_model_output_path=/fsx/home-augustas/logs/unifiedqa-v2-t5-3b-1363200_custom_data_v4_all_20230629_120158_21789",
    "--dataset_name=AugustasM/burns-datasets-VINC-ppo-training-v4",
    "--remove_unused_columns=False",
    "--log_with=tensorboard",
    "--logging_dir=/fsx/home-augustas/ppo_logs",
    "--learning_rate=1.4e-5",
    "--batch_size=1",
    "--mini_batch_size=1",
    "--gradient_accumulation_steps=64",
    "--steps=192",
    "--ppo_epochs=4",
    "--early_stopping=True",
    "--reward_baseline=0.0",
    "--target_kl=0.1",
    "--init_kl_coef=0.2",
    "--adap_kl_ctrl=True",
    "--seed=0",
    "--save_freq=2",
    "--output_dir=/fsx/home-augustas/ppo_runs/test",
])
print(script_args.dataset_name)

# Get PPO config
config = get_ppo_config(script_args)
print(config.total_ppo_epochs)
print(config.project_kwargs)

AugustasM/burns-datasets-VINC-ppo-training-v4
192
{'logging_dir': '/fsx/home-augustas/ppo_logs'}


In [5]:
# Tokenizer
tokenizer = get_tokenizer(script_args.tokenizer_name)

# Dataset for PPO training
train_dataset, prompt_max_len, response_max_len = get_dataset(
    script_args.dataset_name, tokenizer, subsets_to_delete=["piqa"]
)

Loading tokenizer gpt2...


Using pad_token, but it is not set yet.


Loaded tokenizer.

Loading dataset...



Found cached dataset parquet (/admin/home-augustas/.cache/huggingface/datasets/AugustasM___parquet/AugustasM--burns-datasets-VINC-ppo-training-v4-278eadae0cef7ee6/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
Loading cached processed dataset at /admin/home-augustas/.cache/huggingface/datasets/AugustasM___parquet/AugustasM--burns-datasets-VINC-ppo-training-v4-278eadae0cef7ee6/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-bc44c211aecff264.arrow


Deleting subset: piqa


Loading cached processed dataset at /admin/home-augustas/.cache/huggingface/datasets/AugustasM___parquet/AugustasM--burns-datasets-VINC-ppo-training-v4-278eadae0cef7ee6/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-87b3b92eba672a07_*_of_00012.arrow
Loading cached processed dataset at /admin/home-augustas/.cache/huggingface/datasets/AugustasM___parquet/AugustasM--burns-datasets-VINC-ppo-training-v4-278eadae0cef7ee6/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-565481452045ac3d_*_of_00012.arrow



Max prompt length: 550

Max response length: 11

Remaining columns: ['prompt', 'input_ids', 'attention_mask', 'response_len']

Total number of examples: 8615

Processing finished.



In [6]:
# set seed before initializing value head for deterministic eval
set_seed(config.seed)

# Now let's build the model, the reference model, and the tokenizer.
current_device = Accelerator().local_process_index
print(f"Current device: {current_device}\n")

# Get the reward model
reward_model, reward_model_name = get_reward_model(
    script_args.reward_model_output_path, current_device,
)
reward_model_tokenizer = get_tokenizer(reward_model_name)

Current device: 0

The current device is 0.

Loading reward model from allenai/unifiedqa-v2-t5-3b-1363200.
is_bf16_possible=True
Loaded reward model with 2,851,598,336 parameters.
Reward model dtype: torch.bfloat16

Loading reporter from /fsx/home-augustas/VINC-logs/allenai/unifiedqa-v2-t5-3b-1363200/AugustasM/burns-datasets-VINC-v4/festive-feistel/reporters/layer_19.pt
Loaded reporter.

Loading tokenizer allenai/unifiedqa-v2-t5-3b-1363200...
Falling back to slow tokenizer; fast one failed: 'No such file or directory (os error 2)'
Loaded tokenizer.



In [7]:
# Model
model = get_model(script_args.model_name, current_device)

Loading subject model...

is_bf16_possible=True


Loaded subject model with 124,440,577 parameters.
Model dtype: torch.bfloat16



In [8]:
dataset_template_path = "AugustasM/burns-datasets-VINC/first"

dataset_templates = DatasetTemplates(dataset_template_path)
dataset_templates.templates = {
    x.name: x for x in dataset_templates.templates.values()
}
print(len(dataset_templates.templates))

template = list(dataset_templates.templates.values())[0]
template

1


In [9]:
# Optimizer
# TODO: consider whether adding Adafactor back in is a good idea
optimizer = None

# We then build the PPOTrainer, passing the model, the reference model, the tokenizer
ppo_trainer = PPOTrainer(
    config,
    model,
    ref_model=None,
    tokenizer=tokenizer,
    dataset=train_dataset,
    data_collator=collator,
    optimizer=optimizer,
)

In [10]:
# TODO: put this into config
# TODO: check whether there are better settings for this
# We then define the arguments to pass to the `generate` function. These arguments
# are passed to the `generate` function of the PPOTrainer, which is a wrapper around
# the `generate` function of the trained model.
generation_kwargs = {
    "top_k": 0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.pad_token_id,
    "eos_token_id": 100_000, # why is this value like this?
    "pad_to_multiple_of": 8, # TODO: double-check, but this seems to work and to be faster
}

train(
    ppo_trainer=ppo_trainer,
    tokenizer=tokenizer,
    generation_kwargs=generation_kwargs,
    reward_model=reward_model,
    reward_model_tokenizer=reward_model_tokenizer,
    template=template,
    script_args=script_args,
    config=config,
    device=current_device,
)

0it [00:00, ?it/s]You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

1it [00:06,  6.72s/it]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

2it [00:09,  4.28s/it]

...

190it [10:27,  3.41s/it]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

191it [10:31,  3.52s/it]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

192it [10:34,  3.30s/it]


In [11]:
device = torch.cuda.current_device()
print(f"Device: {device}")

gpu_properties = torch.cuda.get_device_properties(device)
print(f"GPU Name: {gpu_properties.name}")
print(f"Total GPU Memory: {gpu_properties.total_memory / 1024**3} GB")
print(f"Allocated GPU Memory: {torch.cuda.memory_allocated(device) / 1024**3} GB")
print(f"Cached GPU Memory: {torch.cuda.memory_reserved(device) / 1024**3} GB")

Device: 0
GPU Name: NVIDIA A100-SXM4-40GB
Total GPU Memory: 39.56402587890625 GB
Allocated GPU Memory: 6.586625099182129 GB
Cached GPU Memory: 6.896484375 GB


In [20]:
import time
from datetime import datetime

def your_function():
    # Your function code here
    time.sleep(3)  # Simulating a delay of 5 seconds

start_time = datetime.now()
your_function()
elapsed_time = datetime.now() - start_time
print(elapsed_time.total_seconds())
elapsed_time = datetime.utcfromtimestamp(
    elapsed_time.total_seconds()
).strftime("%H:%M:%S")

print(f"Duration: {elapsed_time}")

3.003301
Duration: 00:00:03
